In [3]:
import keras
import numpy as np
from keras import layers
import pandas as pd
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout, GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.utils import np_utils, print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as k


In [4]:


data=pd.read_csv(r'C:\Users\Rittik Grignard\Downloads\train_foo.csv')
dataset=np.array(data)
np.random.shuffle(dataset)
x=dataset
y=dataset
x=x[:,0:2500]
y=y[:,0]
x_train=x[0:12000,:]
x_train=x_train/255.
x_test=x[12000:13201,:]
x_test=x_test/255.
x_train=np.asarray(x_train)
x_test=np.asarray(x_test)
y=y.reshape(y.shape[0],1)
y_train=y[0:12000,:]
y_train=y_train.T
y_test=y[12000:13201,:]
y_test=y_test.T

print(x_train.shape[0])

print(x_test.shape[0])
image_x=50
image_y=50

train_y=np_utils.to_categorical(y_train)
test_y=np_utils.to_categorical(y_test)
train_y=np.asarray(train_y)
test_y=np.asarray(test_y)

train_y=train_y.reshape(train_y.shape[1],train_y.shape[2])
test_y=test_y.reshape(test_y.shape[1],test_y.shape[2])
x_train=x_train.reshape(x_train.shape[0],50,50,1)
x_test=x_test.reshape(x_test.shape[0],50,50,1)
print("x train shape:"+str(x_train.shape))
print("x test shape:"+str(x_test.shape))
print(train_y.shape)

12000
1199
x train shape:(12000, 50, 50, 1)
x test shape:(1199, 50, 50, 1)
(12000, 12)


In [5]:
#Building the model
def keras_model(image_x,image_y):
    num_of_classes=12
    model=Sequential()
    model.add(Conv2D(filters=32,kernel_size=(5,5),input_shape=(image_x,image_y,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))
    model.add(Conv2D(64,(5,5),activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5),strides=(5,5),padding='same'))
    model.add(Flatten())
    model.add(Dense(1024,activation='relu'))
    model.add(Dropout(0,6))
    model.add(Dense(num_of_classes,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    filepath="face-rec_256.h5"
    checkpoint1=ModelCheckpoint(filepath,monitor='val_acc',verbose=1, save_best_only=True,mode='max')
    callbacks_list=[checkpoint1]
    return model,callbacks_list


In [6]:
model,callbacks_list=keras_model(image_x,image_y)
model.fit(x_train,train_y,validation_data=(x_test,test_y),epochs=1,batch_size=64,callbacks=callbacks_list)
scores=model.evaluate(x_test,test_y,verbose=0)
print("CNN:Error%.2f%%"%(100-scores[1]*100))
print_summary(model)
model.save('handemo.h5')



Train on 12000 samples, validate on 1199 samples
Epoch 1/1
12000/12000 [==============================] - 68s 6ms/step - loss: 0.1142 - accuracy: 0.9659 - val_loss: 4.8069e-04 - val_accuracy: 1.0000


C:\Users\Rittik Grignard\Anaconda\envs\deeplearning\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


CNN:Error0.00%
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 19, 19, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 102